In [99]:
import datetime
from selenium import webdriver
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.service import Service
import csv

import matplotlib.pyplot as plt

import numpy as np
import pandas as pd

# import plotly.plotly as py
# import plotly.graph_objs as go
# from plotly.offline import iplot, init_notebook_mode

options = Options()
options.add_argument("--headless")

pd.set_option('display.max_rows', 1000)

#service = Service('/Users/henry/Downloads/chromedriver')
driver = webdriver.Chrome(options=options) #service=service, options=options
#This is the path to your Chrome webdriver. Make sure you have the right one for your OS

In [100]:
def get_prop_lines(prop):
    prop.click()
    #fetches all games for a given prop
    games = driver.find_elements_by_class("market-block")

    for game in games:
        game.click()
        driver.implicitly_wait(2)
        players = game.find_elements_by_class_name("player-group")
        wagers = {}
        for player in players:
            name = player.find_element_by_tag_name("b").text
            lines = player.find_element_by_tag_name("option")[0].get_attribute("value")
            odds = player.find_element_by_tag_name("button").text
            wagers = {"name": name, "lines": lines, "odds": odds}

    result = pd.DataFrame(wagers)

    return result

In [105]:
#Main function to run the scraper
def bovScrape_MX_props():
    driver.get('https://www.bovada.lv/sports/soccer/north-america/mexico/liga-mx-apertura')
    driver.implicitly_wait(5)

    #This is the path to the "Assists" button. It will click the button to show Assists
    wait = WebDriverWait(driver, 10)
    league_box = wait.until(EC.presence_of_element_located((By.ID, "playerPropsLeagueFrame")))
    league_frame = league_box.find_element(By.TAG_NAME, "iframe")
    driver.switch_to.frame(league_frame)
    league_layer1 = driver.find_element(By.TAG_NAME, "html")
    league_layer2 = WebDriverWait(league_layer1, 10).until(EC.presence_of_element_located((By.TAG_NAME, "body")))
    league_layer3 = league_layer2.find_element(By.TAG_NAME, "app-cross-game-widgets")
    league_layer4 = league_layer3.find_element(By.TAG_NAME, "form")
    #prop_types = WebDriverWait(league_box, 10).until(EC.presence_of_element_located((By.TAG_NAME, "select")))
    shots = league_layer4.find_elements(By.XPATH, "//option[@value='Shots']")
    sogs = league_layer4.find_elements(By.XPATH, "//option[@value='Shots on Goal']")
    print(shots, sogs)
    #prop_options = prop_types[0].find_elements(By.TAG_NAME, "option")
    shots_lines = get_prop_lines(shots)
    shots_on_target_lines = get_prop_lines(sogs)

    return shots_lines, shots_on_target_lines
    

In [106]:
shots, sogs = bovScrape_MX_props()
shots.head()

[<selenium.webdriver.remote.webelement.WebElement (session="068a1dbcb9d3d65760e109ebfe37fe51", element="6048A438440D1170E277BD3F46876E7C_element_153")>] [<selenium.webdriver.remote.webelement.WebElement (session="068a1dbcb9d3d65760e109ebfe37fe51", element="6048A438440D1170E277BD3F46876E7C_element_154")>]


AttributeError: 'list' object has no attribute 'click'